In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
if path.exists('/opt/bin/nvidia-smi'):
  !pip install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl torchvision
  !pip install dotted pyfastnoisesimd tqdm Pillow==4.0.0 PIL image
  !wget -nc https://warwick.ac.uk/fac/sci/dcs/research/tia/glascontest/download/warwick_qu_dataset_released_2016_07_08.zip -O warick.zip
  !unzip -q -o warick.zip
  !mv 'Warwick QU Dataset (Released 2016_07_08)' warick_data
else:
  print('Select GPU backend')

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from dotted.collection import DottedDict
from pyfastnoisesimd import generate
from glob import glob
import numpy as np
import pickle
import random
from random import uniform, randint
import tqdm

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

def conv(in_c, out_c):
  return nn.Sequential(
    nn.Conv2d(in_c, out_c, 3, padding=1, bias=False),
    nn.ELU(inplace=True),
    nn.BatchNorm2d(out_c),
    nn.Conv2d(out_c, out_c, 3, padding=1, bias=False),
    nn.ELU(inplace=True),
    nn.BatchNorm2d(out_c),
  )

class UNet(nn.Module):
  def __init__(self):
    super(UNet, self).__init__()
    self.down1 = conv(  3,  16) # (  3, 512, 512) --> ( 16, 512, 512)
    self.down2 = conv( 16,  32) # ( 16, 256, 256) --> ( 32, 256, 256)
    self.down3 = conv( 32,  64) # ( 32, 128, 128) --> ( 64, 128, 128)
    self.down4 = conv( 64, 128) # ( 64,  64,  64) --> (128,  64,  64)
    self.down5 = conv(128, 256) # (128,  32,  32) --> (256,  32,  32)
    self.down6 = conv(256, 512) # (256,  16,  16) --> (512,  16,  16)
    self.up1   = conv(768, 256) # (768,  32,  32) --> (256,  32,  32)
    self.up2   = conv(384, 128) # (384,  64,  64) --> (128,  64,  64)
    self.up3   = conv(192,  64) # (192, 128, 128) --> ( 64, 128, 128)
    self.up4   = conv( 96,  32) # ( 32, 256, 256) --> ( 16, 256, 256)
    self.up5   = conv( 48,  16) # ( 32, 512, 512) --> ( 16, 512, 512)
    self.tail  = nn.Conv2d(16, 1, 1)
    self.downpool = nn.MaxPool2d(kernel_size=2)
    self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)

  def forward(self, x):
    x_down_512 = self.down1(x)
    x_down_256 = self.down2(self.downpool(x_down_512))
    x_down_128 = self.down3(self.downpool(x_down_256))
    x_down_64  = self.down4(self.downpool(x_down_128))
    x_down_32  = self.down5(self.downpool(x_down_64))
    x_down_16  = self.down6(self.downpool(x_down_32))
    x_up = self.up1(torch.cat([self.upsample(x_down_16), x_down_32], dim=1))
    x_up = self.up2(torch.cat([self.upsample(x_up), x_down_64], dim=1))
    x_up = self.up3(torch.cat([self.upsample(x_up), x_down_128], dim=1))
    x_up = self.up4(torch.cat([self.upsample(x_up), x_down_256],  dim=1))
    x_up = self.up5(torch.cat([self.upsample(x_up), x_down_512]))
    return self.tail(x_up)

def plot_metrics(metric_logs):
  for metric_log in metric_logs:
    plt.plot(metric_log['epoch'], metric_log['metric'], label=metric_log['label'])
  plt.xlabel('Epochs')
  plt.ylabel('Metric')
  plt.ylim(0, 5)
  plt.legend()
  plt.show()

def dice(pred, targs):
    pred = (pred>0).float()
    return 2. * (pred*targs).sum() / (pred+targs).sum()
  
def dice_loss(input, target):
    smooth = 1.

    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()

    return 1.0 - (((2. * intersection + smooth) /
              (iflat.sum() + tflat.sum() + smooth)))
  
def train(model, epochs=1):
  for e in tqdm.trange(epochs, desc='epochs'):
    metric = 0
    samples_seen = 0
    model.net.train()
    for img, mask in model.loader:
      model.img_cuda.copy_(img)
      del img
      model.mask_cuda.copy_(mask)
      del mask
      model.optimizer.zero_grad()
      prediction = model.net(model.img_cuda)
      loss = (F.binary_cross_entropy_with_logits(prediction, model.mask_cuda) +
              dice_loss(F.sigmoid(prediction), model.mask_cuda)) * 100
      metric += loss.item()
      samples_seen += model.batch_size
      loss.backward()
      model.optimizer.step()
      model.scheduler.step()
    model.train_metric_log['epoch'].append(model.epochs_trained)
    model.train_metric_log['metric'].append(metric / samples_seen)
    
    if model.epochs_trained % model.eval_test == 0:
      metric = 0
      samples_seen = 0
      model.net.eval()
      with torch.no_grad():
        for img, mask in model.loader_test:
          model.img_cuda.copy_(img)
          del img
          model.mask_cuda.copy_(mask)
          del mask
          prediction = model.net(model.img_cuda)
          loss = (F.binary_cross_entropy_with_logits(prediction, model.mask_cuda) +
                  dice_loss(F.sigmoid(prediction), model.mask_cuda)) * 100
          metric += loss.item()
          samples_seen += model.batch_size
      model.test_metric_log['epoch'].append(model.epochs_trained)
      model.test_metric_log['metric'].append(metric / samples_seen)
    model.epochs_trained += 1

def generate_noise():
  return fns.generate(size=[1, 128, 128], noiseType='Perlin',
                      freq=uniform(.01, .075), seed=randint(0, 100000))[0]
    
def interpolate(a, b, f):
  return (a * (1.0 - f)) + (b * f)
    
def train_mixup(model, epochs=1):
  for e in tqdm.trange(epochs, desc='epochs'):
    metric = 0
    samples_seen = 0
    model.net.train()
    for (img_a, mask_a), (img_b, mask_b) in zip(model.loader, model.loader_other):
      mixup_lerp = random.random()
      model.img_cuda.copy_(interpolate(img_a, img_b, mixup_lerp))
      del img_a; del img_b
      model.mask_cuda.copy_(interpolate(mask_a, mask_b, mixup_lerp))
      del mask_a; del mask_b
      model.optimizer.zero_grad()
      prediction = model.net(model.img_cuda)
      loss = (F.binary_cross_entropy_with_logits(prediction, model.mask_cuda) +
              dice_loss(F.sigmoid(prediction), model.mask_cuda)) * 100
      metric += loss.item()
      samples_seen += model.batch_size
      loss.backward()
      model.optimizer.step()
      model.scheduler.step()
    model.train_metric_log['epoch'].append(model.epochs_trained)
    model.train_metric_log['metric'].append(metric / samples_seen)
    
    if model.epochs_trained % model.eval_test == 0:
      metric = 0
      samples_seen = 0
      model.net.eval()
      with torch.no_grad():
        for img, mask in model.loader_test:
          model.img_cuda.copy_(img)
          del img
          model.mask_cuda.copy_(mask)
          del mask
          prediction = model.net(model.img_cuda)
          loss = (F.binary_cross_entropy_with_logits(prediction, model.mask_cuda) +
                  dice_loss(F.sigmoid(prediction), model.mask_cuda)) * 100
          metric += loss.item()
          samples_seen += model.batch_size
      model.test_metric_log['epoch'].append(model.epochs_trained)
      model.test_metric_log['metric'].append(metric / samples_seen)
    model.epochs_trained += 1
  
def weights_init(m):
  with torch.no_grad():
    if isinstance(m, nn.Conv2d):
      torch.nn.init.xavier_uniform_(m.weight)

In [0]:
img, mask = torch.rand([1, 3, 512, 512]), torch.rand([1, 1, 512, 512])

In [0]:
img_mean, img_std = [200.248, 131.253, 199.778], [41.787, 62.667, 32.977]
mask_mean, mask_std = 2.512, 4.168

In [0]:

train_img_files = glob('warick_data/train*[!anno].bmp')
train_mask_files = glob('warick_data/train*anno.bmp')
test_img_files = glob('warick_data/test*[!anno].bmp')
test_mask_files = glob('warick_data/test*anno.bmp')

In [0]:
img_sizes = set(Image.open(file).size for file in files)

sizes_a, sizes_b = [], []
for size_a, size_b in img_sizes:
  sizes_a.append(size_a); sizes_b.append(size_b)
  
sizes_a, sizes_b = np.array(sizes_a), np.array(sizes_b)

sizes_a.mean(), sizes_a.std(), sizes_b.mean(), sizes_b.std()